In [ ]:
%%python module Camunda

from js import XMLHttpRequest
from urllib.parse import urlencode
from IPython.display import display

import json
import os

def http(method, path, params=None, data=None):
    request = XMLHttpRequest.new()
    request.open(method, f"{os.environ['CAMUNDA_ENGINE_API']}{path}?{urlencode(params or {})}", False)
    request.setRequestHeader("X-XSRF-TOKEN", os.environ["CAMUNDA_CSRF_TOKEN"])
    request.setRequestHeader("Content-Type", "application/json")
    request.send(json.dumps(data))
    assert request.status in [200, 204], request.responseText
    return json.loads(request.responseText or "null")

class Camunda:
    ROBOT_LIBRARY_SCOPE = 'SUITE'

    def __init__(self):
        self.process_ids = []
    
    def get_definitions(self):
        params = dict(latest="true")
        results = http("GET", "/process-definition", params)
        return [d["key"] for d in results]
    
    def get_instances(self):
        return http("GET", "/process-instance")
    
    def get_external_tasks(self):
        assert self.process_ids, "No process managed by Camunda library instance is running"
        params = dict(processInstanceId=self.process_ids[-1])
        return http("GET", "/external-task", params)
    
    def complete_external_task(self, topic, variables):
        worker_id = f"{self.__class__.__name__}:{id(self)}"
        data = dict(
            workerId=worker_id,
            maxTasks=1,
            topics=[dict(
                topicName=topic,
                lockDuration=1000
            )]
        )
        external_tasks = http("POST", "/external-task/fetchAndLock", data=data)
        external_tasks = [x for x in external_tasks if x["processInstanceId"] == self.process_ids[-1]]
        assert external_tasks, "No external tasks available"
        for external_task in external_tasks:
            return external_task
    
    def start_instance(self, key):
        data = dict(variables={})
        instance = http("POST", f"/process-definition/key/{key}/start", data=data)
        self.process_ids.append(instance["id"])
        
    def stop_instance(self):
        if self.process_ids:
            params = dict(
                skipCustomListeners="true",
                skipIoMappings="true",
                failIfNotExist="false",
            )
            http("DELETE", f"/process-instance/{self.process_ids.pop()}", params)
        

In [ ]:
*** Settings ***

Library  Camunda

Suite setup  Start instance  ${PROCESS_KEY}
Suite teardown  Stop instance

*** Variables ***

${PROCESS_KEY}  traffic-data-loader

In [ ]:
*** Keywords ***

Number of instances should be
    [Arguments]  ${number}  ${error}
    ${instances}=  Get instances
    ${number of instances}=  Get length  ${instances}
    Should be equal as integers  ${number of instances}  ${number}  ${error}
    
Number of external tasks should be
    [Arguments]  ${number}  ${error}
    ${external tasks}=  Get external tasks
    ${number of external tasks}=  Get length  ${external tasks}
    Should be equal as integers  ${number of external tasks}  ${number}  ${error}

Topic for all externals tasks should be
    [Arguments]  ${topic}  ${error}
    ${external tasks}=  Get external tasks
    FOR  ${external task}  IN  @{external tasks}
        Should be equal  ${external task}[topicName]  ${topic}  ${error}
    END

In [ ]:
*** Test Cases ***

Test process is properly setup and teardown
    Number of instances should be  1  Process should really be running
    
Test start and stop instance
    Start instance  ${PROCESS_KEY}
    Number of instances should be  2  A new process should really be running
    [Teardown]  Stop instance

In [ ]:
*** Test Cases ***

Producer task is the first task
    Number of external tasks should be  1  Process should start with Producer task
    Topic for all externals tasks should be  Produce traffic data work items  Wrong topic

In [ ]:
*** Test Cases ***

WIP
    ${variables}=  Create dictionary
    Complete external task
    ...  Produce traffic data work items
    ...  ${variables}